# Introduction

Issue: we have a ton of unclassified images containing either a document (scanned or in a photo),
a colour photo or a RX medical image (again scanned or in a photo).

Goal: classify all these in the three classes, in order to unload human work.

This is a very simple job for a human eye but with some tricks it can be very easy for a computer too.
We tried to understand, why is it so easy for a human eye? 
How can we differentiate for example between a RX medical image and a colour photo?
Which are the features that can help differentiate them?

Can we do this job without the use of convolutional neural networks?

 
First we create a grid to divide each image into nine squares.

In [1]:
import string

xpix, ypix = 299, 299          # number of x and y pixels into which resizing the original images
white_th, black_th = 235, 20   # threshold to define white and black in grey scale
xsplit, ysplit = 3, 3          # number of splits in x and y direcitons
select = [0,2,4,6,8]           # select squares
suffix = ['_'+i for i in string.ascii_uppercase]  # suffix for features


We create a list with the names of the test images

In [3]:
import os
from os import *

fol = r'./test_images'+os.sep  # define path where test images are stored
included_extensions = ['jpg', 'jpeg', 'tif', 'gif']  # extensions of the images
file = [fn for fn in listdir(fol) if any(fn.endswith(ext) for ext in included_extensions)]  # file names


We define the function to load the image, apply a grid and compute the features

In [16]:
def par_job_dict(ifil): 
    dict_temp = {}
    
    img = Image.open(fol+ifil)
    img = img.resize((xpix,ypix), PIL.Image.ANTIALIAS)
    img = img.convert('RGB')
    img = np.array(img) 
    
    dict_temp['fn'] = str(ifil)

    dict_func = feature_eng_dict(img, '_TOT')
    dict_temp.update(dict_func)
    
    # divide img in subimg
    height, width, channels = img.shape
    xticks = [round(i*1.0/xsplit*width) for i in range(xsplit)] 
    xticks.append(width)
    yticks = [round(i*1.0/xsplit*height) for i in range(ysplit)] 
    yticks.append(height)
    subimg = list(range(len(select)))

    for isel in range(0,len(select)):
        row = int((select[isel])/xsplit)
        col = (select[isel])%xsplit
        subimg[isel] = img[xticks[row]:xticks[row+1],yticks[col]:yticks[col+1]]

        dict_func = feature_eng_dict(subimg[isel], suffix[isel])

        dict_temp.update(dict_func)

    return dict_temp


Now we call the previous function

In [46]:
from joblib import Parallel, delayed
import PIL
from PIL import Image
import numpy as np
from Utilities import *
np.seterr(divide='ignore', invalid='ignore')

num_cores = 8   # number of cores to use
out = Parallel(n_jobs=num_cores, backend = 'multiprocessing', pre_dispatch = '3*n_jobs', verbose=1, 
                   batch_size = 1, max_nbytes='10G',)(delayed(par_job_dict)(ifil) for ifil in file)

df = pd.DataFrame(out)

[Parallel(n_jobs=8)]: Done   3 out of   3 | elapsed:    0.2s finished


In [24]:
feat = ['ncolor','R_G_diff','R_B_diff','G_B_diff','white_perc','black_perc','gray_perc',
        'h_mean','s_mean','v_mean','c_mean','l_mean','s_l_mean','h_median','s_median',
        'v_median','c_median','l_median','s_l_median','h_var','s_var','v_var','c_var',
        'l_var','s_l_var','distq_RG0','distq_RB0','distq_RG1','distq_RB1','distq_RG2',
        'distq_RB2']

cl = ['_TOT'] + suffix[:len(select)]
X = ['fn']

for el in feat:
    c = [el + x  for x in cl]
    X.extend(c)

df = df[X]


We have created a XGBoost model with all the features as predictors
and type as ground truth

In [33]:
import xgboost as xgb

xg_df = xgb.DMatrix(df.iloc[:,1:].values)

model = xgb.Booster(model_file='xgb_17k_final_prob.model') 

In [36]:
prediction = model.predict(xg_df)
predictions_type = np.argmax(prediction,axis=1)

df_output = df[['fn']]
df_output = df_output.join(pd.DataFrame({'predict':predictions_type}))

df_output.to_csv('df_predicted.csv', sep=';', index=False)